In [ ]:
from google.colab import files
uploaded = files.upload()

Saving clean_dataset.csv to clean_dataset.csv


In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('clean_dataset.csv')

In [ ]:
df.isnull().sum()
rvar_list =['Industry', 'Ethnicity', 'Citizen', 'DriversLicense','ZipCode' ]
df_sample1 = df.drop(columns=rvar_list)
cvar_list = ['Gender', 'Married', 'BankCustomer', 'PriorDefault', 'Employed']
nvar_list = ['Age', 'Debt', 'YearsEmployed', 'CreditScore', 'Income']
df_sample1.isnull().sum()

Gender           0
Age              0
Debt             0
Married          0
BankCustomer     0
YearsEmployed    0
PriorDefault     0
Employed         0
CreditScore      0
Income           0
Approved         0
dtype: int64

In [ ]:
df_sample2 = df_sample1.copy()
df_sample2[nvar_list] = (df_sample1[nvar_list] - df_sample1[nvar_list].mean())/df_sample1[nvar_list].std()

df_sample3 = df_sample2.copy()
df_sample3[cvar_list] = df_sample2[cvar_list].astype('category')
df_sample3[nvar_list] = df_sample2[nvar_list].astype('float64')

df_sample4 = df_sample3.copy()
df_sample4 = pd.get_dummies(df_sample3, prefix_sep='_')

# Find mode for each cvar
#mode_value = df[''].mode()
#print(mode_value)

rdummies = ['Gender_1', 'Married_1', 'BankCustomer_1', 'PriorDefault_1', 'Employed_0']
df_sample5 = df_sample4.copy()
df_sample5 = df_sample4.drop(columns=rdummies)
print(df_sample5.columns.values)

df_sample5.to_csv('C:\\Users\\Dioma\\Downloads\\New_clean_dataset.csv')


['Age' 'Debt' 'YearsEmployed' 'CreditScore' 'Income' 'Approved' 'Gender_0'
 'Married_0' 'BankCustomer_0' 'PriorDefault_0' 'Employed_1']


In [ ]:
# Part 5 Data Partition (New a package that can also be used to build prediction models)

from sklearn.model_selection import train_test_split

#import sklearn as sk would import the whole package where as the above code only imports one part
df4partition = df_sample5
testpart_size = 0.2

df_nontestData, df_testData = train_test_split(df_sample5, test_size=0.2, random_state=1)
print(df_testData)

          Age      Debt  YearsEmployed  CreditScore    Income  Approved  \
666 -0.823264  1.404389      -0.589690    -0.493529 -0.195272         0   
520 -0.935404  0.550660      -0.216167    -0.287892 -0.150359         1   
681 -0.310627 -0.755042       0.232061    -0.493529 -0.092203         0   
23  -0.345197  1.956801       0.257460    -0.287892 -0.193160         1   
65   0.223932 -0.646569      -0.204214    -0.287892  9.401468         1   
..        ...       ...            ...          ...       ...       ...   
17  -0.696791  0.224234       0.282860     1.562841 -0.148248         1   
180  1.362188 -0.897665       3.817882     3.619210  2.683750         1   
259 -0.591397 -0.604384      -0.615090    -0.493529 -0.195272         0   
345  2.633663  0.450221      -0.664395    -0.493529 -0.192968         0   
45   1.923728  0.400002       0.120004     1.768477 -0.140762         1   

     Gender_0  Married_0  BankCustomer_0  PriorDefault_0  Employed_1  
666         1          0    

In [ ]:
# Part 6 Logistic regression

from sklearn.linear_model import LogisticRegression, LogisticRegressionCV

DV = 'Approved'
y = df_nontestData[DV]
x = df_nontestData.drop(columns = [DV])

#penalty level
alpha = 10

clf = LogisticRegression(C=1/alpha, penalty = 'l1', solver = 'saga', max_iter = 200, random_state=1).fit(x,y)

#display estimated coefficients of model candidate
def summary_coef(model_object):
  n_predictors = x.shape[1]
  model_coef = pd.DataFrame(model_object.coef_.reshape(1,n_predictors), columns = x.columns.values)
  model_coef['Intercept'] = model_object.intercept_
  return model_coef.transpose()

print(summary_coef(clf))

                       0
Age             0.000000
Debt            0.000000
YearsEmployed   0.181336
CreditScore     0.678343
Income          0.234312
Gender_0        0.000000
Married_0       0.000000
BankCustomer_0  0.000000
PriorDefault_0 -2.679354
Employed_1      0.000000
Intercept       0.958426


In [ ]:
# Run Logistic regression with k-fold cross validation with k=5
# Placeholder variable: kfolds
kfolds = 5

# Here we specify within which range of the penalty levels we will search for the optimal penalty level,
# i.e., the level that leads to the best model candidate
# We search the optimal alpha within [min_alpha, max_alpha]
min_alpha = 0.001
max_alpha = 100

# We further discretize the continuous alpha range [min_alpha, max_alpha] into n individual points of alpha
# We train n model candidates each of which corresponds to one individual alpha point
n_candidates = 1000

import numpy as np

# We store the list of individual alpha points into alpha_list
alpha_list = list(np.linspace(min_alpha, max_alpha, num=n_candidates))

# C_list is the element-wise inverse of alpha_list. It is required as one of the paramater values for LogisticRegressionCV
C_list = list(1/np.linspace(min_alpha, max_alpha, num=n_candidates))

# Set n_jobs to be -1 to run LogisticRegressionCV on all CPU cores.
clf_optimal = LogisticRegressionCV(Cs=C_list, cv=kfolds, penalty='l1', solver='saga', max_iter=2000, random_state=1, n_jobs=-1).fit(x,y)

# Display the estimated coefficients of the final selected model
print(summary_coef(clf_optimal))

# Display the optimal alpha that yields the final selected model (the best model candidate)
print(1/clf_optimal.C_)

# calculate error rate  over test partition
# the actual values of DV in test partition
y_test_actual = df_testData[DV]
#predictor values in the test partition
x_test = df_testData.drop(columns=[DV])
#predicted values of DV in test partition
y_test_predicted = clf_optimal.predict(x_test)

from sklearn import metrics
print(metrics.confusion_matrix(y_test_actual, y_test_predicted))

#to get accuracy use this code (ex. getting .96 means error rate of 4%)
print(clf_optimal.score(x_test, y_test_actual))

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


                       0
Age            -0.184025
Debt           -0.088207
YearsEmployed   0.347703
CreditScore     0.697204
Income          1.862477
Gender_0        0.035825
Married_0       1.314176
BankCustomer_0 -1.730639
PriorDefault_0 -3.257112
Employed_1      0.314951
Intercept       1.258711
[0.3012973]
[[61 14]
 [ 3 60]]
0.8768115942028986


In [ ]:
# Part7 Score the new data

# Upload the new data file from the local drive to Colab
from google.colab import files
uploaded = files.upload()

Saving cleaned_dataset Final.csv to cleaned_dataset Final (1).csv


In [ ]:
# Load the data from the new data file to a Python dataframe df_newdata
df_newdata = pd.read_csv('cleaned_dataset Final.csv')

# Check if there is any regular missing value in df_newdata
print(df_newdata.isnull().sum())

Age               0
Debt              0
YearsEmployed     0
CreditScore       0
Income            0
Approved          0
Gender_0          0
Married_0         0
BankCustomer_0    0
PriorDefault_0    0
Employed_1        0
dtype: int64


In [ ]:
# Generate the categorical predictor list
Original_DV = 'Approved'
cpredictor_list = cvar_list.copy()

df_newdata_sample1 = df_newdata.drop(columns = [Original_DV])


# Derive the sample mean and the sample standard deviation of the numerical variables in the historical data
historical_sample_mean = df_sample1[nvar_list].mean()
historical_sample_std = df_sample1[nvar_list].std()

# Use the historical sample mean and historical sample standard deviation to standardize the new data
df_newdata_sample2 = df_newdata_sample1.copy()
df_newdata_sample2[nvar_list] = (df_newdata_sample1[nvar_list] - historical_sample_mean[nvar_list])/historical_sample_std[nvar_list]

# Code the categorical variables in the new data
df_newdata_sample3 = pd.get_dummies(df_newdata_sample2, prefix_sep='_')

# Fix the inconsistency between the predictors of the new data and the predictors required by the final selected model
df_newdata_sample4 = df_newdata_sample3.copy()

print(df_newdata_sample4.columns.values)

['Age' 'Debt' 'YearsEmployed' 'CreditScore' 'Income' 'Gender_0'
 'Married_0' 'BankCustomer_0' 'PriorDefault_0' 'Employed_1']


In [ ]:
clf_optimal = LogisticRegressionCV(Cs=C_list, cv=kfolds, penalty='l1', solver='saga', max_iter=2000, random_state=1, n_jobs=-1).fit(x,y)

# Score the input data using the model carried by the model object clf_optimal
predicted_ApprovalStatus = clf_optimal.predict(df_newdata_sample4)
predicted_ApprovalStatus_prob = clf_optimal.predict_proba(df_newdata_sample4)[:, 1]

# Print the predicted probability for the input data
print(predicted_ApprovalStatus)
print(predicted_ApprovalStatus_prob)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


[1 1 1 1 1 1 1 1 1 1 0 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 0 0 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 1 1 1 

In [ ]:
print(predicted_ApprovalStatus.shape)
print(predicted_ApprovalStatus_prob.shape)

(690,)
(690,)
